In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

C:\Users\coolt\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [39]:
df = pd.DataFrame()
url = "https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW"
result = requests.get(url)
doc = BeautifulSoup(result.text, "html.parser")
movie_table = doc.find_all("table")

movie_data = []
for row in movie_table[0].find_all("tr")[1:]:  # Skip the header row
    cells = row.find_all("td")
    current_title = cells[1].text
    current_gross = cells[2].text[1:]
    current_year = cells[3].text
    
    
    movie_data.append({"Title" : current_title,
                       "Lifetime Gross" : current_gross,
                       "Year" : current_year,
                      }
                     )
                       
temp = pd.DataFrame(movie_data)
df = pd.concat([temp, df])

for i in range(200,1000,200):
    url = "https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW&offset="+str(i)
    result = requests.get(url)
    doc = BeautifulSoup(result.text, "html.parser")
    movie_table = doc.find_all("table")
    movie_data = []
    for row in movie_table[0].find_all("tr")[1:]:  # Skip the header row
        cells = row.find_all("td")
        current_title = cells[1].text
        current_gross = cells[2].text[1:]
        current_year = cells[3].text


        movie_data.append({"Title" : current_title,
                           "Lifetime Gross" : current_gross,
                           "Year" : current_year,
                          }
                         )

    temp = pd.DataFrame(movie_data, index = [i for i in range(i,i+200,1)])
    df = pd.concat([temp, df])
    
revenue_data = df

### Import Kaggle Datasets

In [4]:
import opendatasets as od
import os
movie_industry = "https://www.kaggle.com/datasets/danielgrijalvas/movies/data"
imdb2000 = "https://www.kaggle.com/datasets/thegoanpanda/imdb-top-2000-movies"

od.download(movie_industry)
od.download(imdb2000)

Skipping, found downloaded files in ".\movies" (use force=True to force download)
Skipping, found downloaded files in ".\imdb-top-2000-movies" (use force=True to force download)


In [5]:
movies = '.\movies'
imdb = '.\imdb-top-2000-movies'
os.listdir(movies)
os.listdir(imdb)

['IMDb Top 2000 Movies.csv']

In [9]:
movie_industry = pd.read_csv('movies.csv')
imdb2000 = pd.read_csv(r"C:\Users\coolt\Desktop\IMDb Top 2000 Movies.csv", encoding_errors ='replace')

In [21]:
movie_industry["Title"] = movie_industry["name"]
movie_industry = movie_industry.drop("name", axis = 1)
movie_industry.head()

,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,Title
0,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,The Shining
1,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,The Blue Lagoon
2,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,Star Wars: Episode V - The Empire Strikes Back
3,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0,Airplane!
4,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0,Caddyshack


In [33]:
print(len(movie_industry))

7668


In [ ]:
imdb2000["Title"] = imdb2000["Movie Name"]
imdb2000 = imdb2000.drop("Movie Name", axis = 1)
imdb2000.head()

In [29]:
print(len(imdb2000))

2000


In [37]:
kaggle_data = pd.merge(imdb2000, movie_industry, how = "inner", on = "Title")

In [41]:
df = pd.merge(kaggle_data, revenue_data, how = "inner", on = "Title")

In [43]:
df

,Movie Rank,Year of Release,IMDB Rating,Title,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,Lifetime Gross,Year
0,14,1994,8.9,Pulp Fiction,R,Crime,1994,"October 14, 1994 (United States)",8.9,1900000.0,Quentin Tarantino,Quentin Tarantino,John Travolta,United States,8000000.0,2.139288e+08,Miramax,154.0,"213,928,762",1994
1,18,1993,9.0,Schindler's List,R,Biography,1993,"February 4, 1994 (United States)",8.9,1200000.0,Steven Spielberg,Thomas Keneally,Liam Neeson,United States,22000000.0,3.221612e+08,Universal Pictures,195.0,"322,161,245",1993
2,77,2010,8.3,Toy Story 3,G,Animation,2010,"June 18, 2010 (United States)",8.2,776000.0,Lee Unkrich,John Lasseter,Tom Hanks,United States,200000000.0,1.066971e+09,Walt Disney Pictures,103.0,"1,067,316,101",2010
3,123,I) (2015,8.1,Inside Out,PG,Animation,2015,"June 19, 2015 (United States)",8.1,641000.0,Pete Docter,Pete Docter,Amy Poehler,United States,175000000.0,8.588480e+08,Pixar Animation Studios,95.0,"858,852,998",2015
4,126,2018,8.4,Spider-Man: Into the Spider-Verse,PG,Animation,2018,"December 14, 2018 (United States)",8.4,411000.0,Bob Persichetti,Phil Lord,Shameik Moore,United States,90000000.0,3.755408e+08,Sony Pictures Entertainment (SPE),117.0,"384,298,736",2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,955,2006,8.0,Casino Royale,PG-13,Action,2006,"November 17, 2006 (United States)",8.0,594000.0,Martin Campbell,Neal Purvis,Daniel Craig,United Kingdom,150000000.0,6.165029e+08,Columbia Pictures,144.0,"616,505,162",2006
249,959,2018,8.2,Green Book,PG-13,Biography,2018,"November 16, 2018 (United States)",8.2,414000.0,Peter Farrelly,Nick Vallelonga,Viggo Mortensen,United States,23000000.0,3.217527e+08,Participant,130.0,"321,752,656",2018
250,961,2010,8.0,Black Swan,R,Drama,2010,"December 17, 2010 (United States)",8.0,716000.0,Darren Aronofsky,Mark Heyman,Natalie Portman,United States,13000000.0,3.293980e+08,Fox Searchlight Pictures,108.0,"329,398,046",2010
251,968,1978,7.7,Halloween,R,Horror,2007,"August 31, 2007 (United States)",6.1,111000.0,Rob Zombie,Rob Zombie,Scout Taylor-Compton,United States,15000000.0,8.046095e+07,Dimension Films,109.0,"259,939,869",2018
